In [107]:
import os
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import clear_output

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.multibody import inverse_kinematics
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    DiagramBuilder,
    InverseKinematics,
    LogVectorOutput,
    MeshcatVisualizer,
    Parser,
    PiecewiseQuaternionSlerp,
    PiecewisePolynomial,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    Sphere,
    StartMeshcat,
)


from manipulation import running_as_notebook, ConfigureParser
from manipulation.station import MakeHardwareStation, load_scenario


from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_robot_painter import TestRobotPainter
from manipulation.meshcat_utils import AddMeshcatTriad, MeshcatPoseSliders
from manipulation.scenarios import AddIiwaDifferentialIK
from manipulation.scenarios import AddIiwa, AddShape, AddWsg
from manipulation.station import MakeHardwareStation, load_scenario

In [46]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [110]:
def run_simulation():
    builder = DiagramBuilder()
    time_step = 0.001
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    Parser(plant).AddModels("model.sdf")
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa, welded=True)
    plant.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    logger = LogVectorOutput(plant.get_state_output_port(), builder)

    diagram = builder.Build()
    simulator = Simulator(diagram)

    # meshcat.StartRecording()

    context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    gripper = plant.GetBodyByName("body")
    initial_pose = plant.EvalBodyPoseInWorld(plant_context, gripper)

    rpy = RollPitchYaw([np.pi, 0, np.pi/2])
    goal_rotation = RotationMatrix(rpy)
    goal_position = np.array([0.91, 0, 0.24])
    goal_pose = RigidTransform(goal_rotation, goal_position)

    ## Interpolate Pose for entry.
    def make_gripper_orientation_trajectory():
        traj = PiecewiseQuaternionSlerp()
        traj.Append(0.0, initial_pose.rotation())
        traj.Append(5.0, goal_pose.rotation())
        return traj

    def make_gripper_position_trajectory():
        traj = PiecewisePolynomial.FirstOrderHold(
            [0.0, 5.0],
            np.vstack(
                [
                    [initial_pose.translation()],
                    [goal_pose.translation()],
                ]
            ).T,
        )
        return traj

    entry_traj_rotation = make_gripper_orientation_trajectory()
    entry_traj_translation = make_gripper_position_trajectory()

    def InterpolatePosePickup(t):
        return RigidTransform(
            RotationMatrix(entry_traj_rotation.orientation(t)),
            entry_traj_translation.value(t),
        )

    t_lst = np.linspace(0, 5, 10)
    pose_lst = []
    for t in t_lst:
        AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePosePickup(t), opacity=0.2)
        pose_lst.append(InterpolatePosePickup(t))
    
    print(pose_lst)

    # simulator.AdvanceTo(3 if running_as_notebook else 0.1)
    # meshcat.PublishRecording()

# These are the default values that MultibodyPlant picked (on Nov 16, 2022) for
# this system.
run_simulation()

[RigidTransform(
  R=RotationMatrix([
    [-1.6653345369377348e-16, 0.23924932921398206, -0.9709581651495907],
    [1.0, 5.551115123125783e-17, -1.1102230246251565e-16],
    [1.1102230246251565e-16, -0.9709581651495907, -0.23924932921398212],
  ]),
  p=[0.46561695565556044, 1.7009942981074756e-16, 0.6793215789060888],
), RigidTransform(
  R=RotationMatrix([
    [-5.551115123125783e-17, 0.3795241867847802, -0.9251818154532391],
    [1.0, -5.551115123125783e-17, 0.0],
    [0.0, -0.9251818154532391, -0.3795241867847803],
  ]),
  p=[0.5149928494716093, 1.5119949316510893e-16, 0.6305080701387455],
), RigidTransform(
  R=RotationMatrix([
    [-1.6653345369377348e-16, 0.5115358502574456, -0.8592619355594616],
    [1.0000000000000002, -1.6653345369377348e-16, 0.0],
    [0.0, -0.8592619355594616, -0.5115358502574459],
  ]),
  p=[0.5643687432876581, 1.3229955651947033e-16, 0.5816945613714024],
), RigidTransform(
  R=RotationMatrix([
    [1.1102230246251565e-16, 0.6324100943876448, -0.77463376670

In [47]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (table_top.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
table_top_sdf_file = os.path.join(temp_dir, "table_top.sdf")
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">

  <model name="table_top">
    <link name="table_top_link">
      <inertial>
        <mass>18.70</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1.0e6</drake:hydroelastic_modulus>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="table_top_center">
      <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(table_top_sdf_file, "w") as f:
    f.write(table_top_sdf)

In [6]:
# home_dir = "/workspaces/64212-Final-Project/"
home_dir = "/workspaces/manip-final-project/"
tennisbottle_file = home_dir + "tennisbottle.sdf"
tennisball_file = home_dir + "tennisball.sdf"
iiwa_file = home_dir + "iiwa14_no_collision.sdf"



It is very helpful to visualize and check your model prior to putting it into simulation. Run the below cell to do so. 

Under the `Scene > drake` dropdown in MeshCat, you can enable viewing 'illustration', 'inertia', and 'proximity' to see the visual model, inertia matrix, and collision model, respectively. You can also adjust the poses of the links in the sliders. 

In [11]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(tennisbottle_file)
visualizer.Run(loop_once=not running_as_notebook)

INFO:drake:Click 'Stop Running' or press Esc to quit


Now, we'll create the simulation with your model. The simulation is replayable in MeshCat. A YCB sugar box has been added to the simulation as a reference, but you may comment it out and remove it.

You will have to replace the link specified in the model directives based on your file. You may have to adjust the starting pose of your model, physical properties of your model, and/or the size of the table (defined at the top of the file) in order to not have your object fall through the world.

In [203]:
def create_scene(table, ball, bottle, iiwa, bouncingball):
    scenario_data = "directives:"
    if table:
        scenario_data += f"""
- add_model:
    name: table_top
    file: file://{table_top_sdf_file}
- add_weld:
    parent: world
    child: table_top::table_top_center
"""
    if ball:
        scenario_data += f"""
- add_model:
    name: tennisball
    file: file://{tennisball_file}
    default_free_body_pose:
        tennisball: # Change here!
            translation: [0, 0, 0.05]
            rotation: !Rpy {{ deg: [0, 0, 0] }}
"""
    if bottle:
        scenario_data += f"""
- add_model:
    name: tennisbottle
    file: file://{tennisbottle_file}
    default_free_body_pose:
        tennisbottle: # Change here!
            translation: [0, 0, 0]
            rotation: !Rpy {{ deg: [0, 0, 0] }}
"""
    if iiwa:
        scenario_data += f"""
- add_model:
    name: iiwa
    file: file://{iiwa_file}
"""
    if bouncingball:
        scenario_data += f"""
    - add_model:
        name: ball
        file: file:///workspaces/64212-Final-Project/bouncingball.sdf
"""
    print(scenario_data)

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()

table = True
ball = True
bottle = True
iiwa = False
bouncingball = False
create_scene(table, ball, bottle, iiwa, bouncingball)

directives:
- add_model:
    name: table_top
    file: file:///tmp/robotlocomotion_drake_ajv4tD/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center

- add_model:
    name: tennisball
    file: file:///workspaces/manip-final-project/tennisball.sdf
    default_free_body_pose:
        tennisball: # Change here!
            translation: [0, 0, 0.05]
            rotation: !Rpy { deg: [0, 0, 0] }

- add_model:
    name: tennisbottle
    file: file:///workspaces/manip-final-project/tennisbottle.sdf
    default_free_body_pose:
        tennisbottle: # Change here!
            translation: [0, 0, 0]
            rotation: !Rpy { deg: [0, 0, 0] }



==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



In [30]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
visualizer.AddModels(url="package://manipulation/mobile_iiwa14_primitive_collision.urdf")
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3acdc8f7-6e42-4688-bf74-f11d9c6c98d8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>